### Maestría: Computación de Alto Desempeño

##### Autor: **Jean Paul Rodríguez**
##### Fecha: **18 de noviembre 2025**
##### Tema: **Procesamiento de Alto Volumen de Datos con el ecosistema Apache Spark**

# Procesamiento de Datos a Gran Escala

<p><strong>Objetivo: </strong> El objetivo de este cuaderno es aprender sentencias pyspark para el preprocesamiento de los datos:</p>

En este notebook se hace un recorrido por las técnicas y procesamientos más comunes que uno hace cuando trabaja con datos en Spark: limpiar, arreglar, transformar y normalizar.

Se muestran jemplos simples pero que sirven igual con datasets gigantes

In [1]:
import findspark

findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

### Arrancando Spark

En esta parte simplemente encendemos Spark con una configuración personalizada

cuántos cores usar, cuánta memoria, y a qué master conectarnos

Se deja listo el entorno para empezar a trabajar

In [2]:
# Crear sesión 

config = (
    SparkConf()
        .set("spark.scheduler.mode", "FAIR")
        .set("spark.executor.cores", "1")
        .set("spark.executor.memory", "4g")
        .set("spark.cores.max", "4")
        #.setMaster("spark://10.43.100.119:8080")
        .setMaster("spark://10.43.100.119:7077")
    )
config.setAppName("hpcspark_jean")
spark = SparkSession.builder.config(conf=config).getOrCreate()

SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)
contextoSpark = spark.sparkContext.getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/17 16:09:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/17 16:09:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Identificación y tratamiento de valores faltantes

Se crea un DataFrame con valores faltantes

Aquí armamos un DataFrame con varios None metidos por ahí

Se usará para practicar cómo detectar y tratar nulos sin usar un dataset real

In [3]:
df = spark.createDataFrame(
[
('Store 1',1,448),
('Store 1',2,None),
('Store 1',3,499),
('Store 1',44,432),
(None,None,None),
('Store 2',1,355),
('Store 2',1,355),
('Store 2',None,345),
('Store 2',3,387),
('Store 2',4,312),
],
['Store','WeekInMonth','Revenue']
)

Indentificación

### Buscando valores nulos

Primero miramos qué filas tienen Revenue = NULL

Luego hacemos un conteo por cada columna para saber cuántos nulos tiene cada una

In [4]:
df.filter(df.Revenue.isNull()).show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          2|   NULL|
|   NULL|       NULL|   NULL|
+-------+-----------+-------+



Contando nulos por columna

In [5]:
from pyspark.sql.functions import count, when, isnull

df.select(
[count(when(isnull(c), c)).alias(c) for c in df.columns]
).show()

+-----+-----------+-------+
|Store|WeekInMonth|Revenue|
+-----+-----------+-------+
|    1|          2|      2|
+-----+-----------+-------+



Eliminado registros con valores faltantes

### Eliminando filas con nulos

Probamos dos formas distintas:

dropna() → elimina filas apenas tengan un null

In [6]:
df2 = df.dropna()
df2.show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          3|    499|
|Store 1|         44|    432|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



dropna('all') → solo borra filas que estén completamente vacías

In [7]:
df2 = df.dropna('all')
df2.show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|   NULL|
|Store 1|          3|    499|
|Store 1|         44|    432|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|       NULL|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



There is one important thing to note about fillna – it’ll only do the exchange
operation for matching column types. So if you use a numeric value for a string column
or the other way around, it won’t work.

Sustituyendo por un valor:

### Rellenando valores faltantes (fillna)

Acá empezamos a rellenar los valores faltantes

Rellenar con 0 en todas las columnas

Rellenar solo en columnas específicas

Rellenar cada columna con un valor distinto usando un diccionario

In [8]:
df.fillna(0).show()
df.fillna(0, ['Revenue']).show()
df.fillna({'WeekInMonth' : 2, 'Revenue' : 3}).show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|      0|
|Store 1|          3|    499|
|Store 1|         44|    432|
|   NULL|          0|      0|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|          0|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|      0|
|Store 1|          3|    499|
|Store 1|         44|    432|
|   NULL|       NULL|      0|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|       NULL|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|      3|
|Store 1

Sustituyendo con la media

Se calcula la media de la columna Revenue y se usa ese valor para hacer el fillna

In [9]:
from pyspark.sql.functions import mean
df.select(mean(df.Revenue)).show()

+------------+
|avg(Revenue)|
+------------+
|     391.625|
+------------+



In [10]:
df.fillna(391.625, ['Revenue']).show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|    391|
|Store 1|          3|    499|
|Store 1|         44|    432|
|   NULL|       NULL|    391|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|       NULL|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



## Eliminando duplicados

Mostramos dos ejemplos:

Quitar duplicados completos de la tabla

In [11]:
df.dropDuplicates().show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 2|          4|    312|
|Store 2|          1|    355|
|Store 2|          3|    387|
|Store 1|         44|    432|
|Store 1|          3|    499|
|Store 1|          1|    448|
|Store 2|       NULL|    345|
|   NULL|       NULL|   NULL|
|Store 1|          2|   NULL|
+-------+-----------+-------+



Quitar duplicados basados solo en ciertas columnas

In [12]:
df.dropDuplicates(['Store','WeekInMonth']).show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 2|          3|    387|
|Store 2|          1|    355|
|Store 2|       NULL|    345|
|Store 2|          4|    312|
|Store 1|          2|   NULL|
|Store 1|          3|    499|
|Store 1|          1|    448|
|   NULL|       NULL|   NULL|
|Store 1|         44|    432|
+-------+-----------+-------+



## Eliminando columnas

Acá literalmente borramos columnas que no se necesitan

Se pueden borrar una o varias columnas

In [13]:
df.drop('Revenue').show()
df.drop('Revenue','Store').show()

+-------+-----------+
|  Store|WeekInMonth|
+-------+-----------+
|Store 1|          1|
|Store 1|          2|
|Store 1|          3|
|Store 1|         44|
|   NULL|       NULL|
|Store 2|          1|
|Store 2|          1|
|Store 2|       NULL|
|Store 2|          3|
|Store 2|          4|
+-------+-----------+

+-----------+
|WeekInMonth|
+-----------+
|          1|
|          2|
|          3|
|         44|
|       NULL|
|          1|
|          1|
|       NULL|
|          3|
|          4|
+-----------+



## Identificando y resolviendo valores inconsistentes

Exploramos el df, se hace un show para ver sus columnas y sus filas

In [14]:
df.show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|   NULL|
|Store 1|          3|    499|
|Store 1|         44|    432|
|   NULL|       NULL|   NULL|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|       NULL|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



Usamos describe() para ver stats como media, mínima, máximo

In [15]:
df.filter(df.Store == 'Store 1').describe().show()

[Stage 32:======================================>                   (2 + 1) / 3]

+-------+-------+-----------------+-----------------+
|summary|  Store|      WeekInMonth|          Revenue|
+-------+-------+-----------------+-----------------+
|  count|      4|                4|                3|
|   mean|   NULL|             12.5|459.6666666666667|
| stddev|   NULL|21.01586702153082|34.99047489436709|
|    min|Store 1|                1|              432|
|    max|Store 1|               44|              499|
+-------+-------+-----------------+-----------------+



Esto dará el valor en un cuantil dado, en el intervalo de 0 a 1. Por lo tanto, si establece el segundo argumento en 0.0, obtendrá el valor más bajo para la columna. Con 1.0 obtienes el valor más alto. En el medio tienes la mediana, que es lo que se está buscando:

Luego usamos approxQuantile() para sacar valores como la mediana sin matar la memoria

Esto nos ayuda a darnos una idea de si algo está fuera de lugar

In [16]:
print(df.approxQuantile('Revenue', [0.5], 0))

[355.0]


## Pivot

A veces, desea cambiar sus datos de filas a columnas. La función se llama pivotar y está disponible en Pyspark.

Básicamente, estás rotando los datos alrededor de un eje determinado, de ahí el nombre.

En este caso, ese eje son los datos en una de sus columnas.

In [17]:
df_pivoted = df.groupBy('WeekInMonth').pivot('Store').sum('Revenue').orderBy('WeekInMonth')
df_pivoted.show()

+-----------+----+-------+-------+
|WeekInMonth|null|Store 1|Store 2|
+-----------+----+-------+-------+
|       NULL|NULL|   NULL|    345|
|          1|NULL|    448|    710|
|          2|NULL|   NULL|   NULL|
|          3|NULL|    499|    387|
|          4|NULL|   NULL|    312|
|         44|NULL|    432|   NULL|
+-----------+----+-------+-------+



El pivot es muy útil cuando se quiere ver métricas por categoría (por ejemplo, por tienda)

Acá se agrupa por store y semana y se hace la suma de revenue para esos grupos

In [18]:
(df
.groupBy('Store','WeekInMonth')
.sum('Revenue')
.orderBy('WeekInMonth')).show()

+-------+-----------+------------+
|  Store|WeekInMonth|sum(Revenue)|
+-------+-----------+------------+
|Store 2|       NULL|         345|
|   NULL|       NULL|        NULL|
|Store 2|          1|         710|
|Store 1|          1|         448|
|Store 1|          2|        NULL|
|Store 2|          3|         387|
|Store 1|          3|         499|
|Store 2|          4|         312|
|Store 1|         44|         432|
+-------+-----------+------------+



Acá deshacemos el pivot usando stack, así nos queda el df como se tenía antes

In [19]:
(df_pivoted.withColumnRenamed('Store 1','Store1')
        .withColumnRenamed('Store 2','Store2')
        .selectExpr('WeekInMonth',"stack(2, 'Store 1', Store1, 'Store 2', Store2) as (Store,Revenue)").show())

+-----------+-------+-------+
|WeekInMonth|  Store|Revenue|
+-----------+-------+-------+
|       NULL|Store 1|   NULL|
|       NULL|Store 2|    345|
|          1|Store 1|    448|
|          1|Store 2|    710|
|          2|Store 1|   NULL|
|          2|Store 2|   NULL|
|          3|Store 1|    499|
|          3|Store 2|    387|
|          4|Store 1|   NULL|
|          4|Store 2|    312|
|         44|Store 1|    432|
|         44|Store 2|   NULL|
+-----------+-------+-------+



## Explode

Hay otra situación con la que te encontrarás de vez en cuando. A veces llegan varios puntos de datos juntos en una columna. Esto usual cuando JSON es el formato de origen.

Puede resolver este problema utilizando el comando de Explode. Tomará la cadena con varios valores y los colocará en una fila cada uno.

In [20]:
from pyspark.sql.functions import explode

df = spark.createDataFrame([
(1, ['Rolex','Patek','Jaeger']),
(2, ['Omega','Heuer']),
(3, ['Swatch','Rolex'])],
('id','watches'))
(df.withColumn('watches',explode(df.watches))).show()

+---+-------+
| id|watches|
+---+-------+
|  1|  Rolex|
|  1|  Patek|
|  1| Jaeger|
|  2|  Omega|
|  2|  Heuer|
|  3| Swatch|
|  3|  Rolex|
+---+-------+



## Normalización

Para este ejercicio se estará utilizando el conjunto de datos Iris, la cual es una fuente en línea en formato CSV (valores separados por coma).

<p> Este set de datos posee diferentes medidas sobre la planta Iris y es famosamente utilizado como ejemplo en analítica de datos:
  </p>
Se utiliza este conjunto para ejemplificar la creación de clusters:

<ul>
  <li>descripción: <a href="https://archive.ics.uci.edu/ml/datasets/Iris" target="_blank">https://archive.ics.uci.edu/ml/datasets/Iris</a></li>
  <li>fuente de datos: <a href="https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data" target="_blank">https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data</a></li>
    <li>tipo de datos: csv</li>

In [21]:
!pip install scikit-learn

In [22]:
import pandas as pd
from sklearn.datasets import load_iris

# cargar iris con pandas
iris = load_iris()
pdf = pd.DataFrame(iris.data, columns=iris.feature_names)
pdf["target"] = iris.target

# convertir a Spark
df = spark.createDataFrame(pdf)

# guardar como parquet usando Spark (NO necesita pyarrow)
df.write.mode("overwrite").parquet("iris.parquet")

Leemos el df público de "iris"

In [23]:
irisDF = spark.read.parquet("iris.parquet")
irisDF.show(5)

+-----------------+----------------+-----------------+----------------+------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|
+-----------------+----------------+-----------------+----------------+------+
|              4.9|             3.6|              1.4|             0.1|     0|
|              4.4|             3.0|              1.3|             0.2|     0|
|              5.1|             3.4|              1.5|             0.2|     0|
|              5.0|             3.5|              1.3|             0.3|     0|
|              4.5|             2.3|              1.3|             0.3|     0|
+-----------------+----------------+-----------------+----------------+------+
only showing top 5 rows



Primero convertimos las columnas numéricas en vector para hacer la normalización

Luego la escalamos para que sus valores queden entre 5 y 10

Esto deja los datos en un rango uniforme

In [24]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler

# Convertir columna numérica a vector
assembler = VectorAssembler(
    inputCols=["sepal width (cm)"],
    outputCol="sepal_width_vec"
)

iris_vec = assembler.transform(irisDF)

# Escalar
scaler = MinMaxScaler(
    min=5,
    max=10,
    inputCol="sepal_width_vec",
    outputCol="sepal_width_scaled"
)

scaler_model = scaler.fit(iris_vec)
scaled_df = scaler_model.transform(iris_vec)

scaled_df.show(truncate=False)

+-----------------+----------------+-----------------+----------------+------+---------------+-------------------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|sepal_width_vec|sepal_width_scaled |
+-----------------+----------------+-----------------+----------------+------+---------------+-------------------+
|4.9              |3.6             |1.4              |0.1             |0     |[3.6]          |[8.333333333333332]|
|4.4              |3.0             |1.3              |0.2             |0     |[3.0]          |[7.083333333333333]|
|5.1              |3.4             |1.5              |0.2             |0     |[3.4]          |[7.916666666666666]|
|5.0              |3.5             |1.3              |0.3             |0     |[3.5]          |[8.125]            |
|4.5              |2.3             |1.3              |0.3             |0     |[2.3]          |[5.625]            |
|4.4              |3.2             |1.3              |0.2             |0     |[3

Ahora normalizamos pero usando media 0 y desviación estándar 1

Este formato es muy común en machine learning
especialmente cuando no se quiere que columnas con rangos grandes tengan más importancia que otras

Esto nivela el campo para que la naturaleza y rango de las variables no impacte en su relevancia

In [25]:
from pyspark.ml.feature import StandardScaler

std_scaler = StandardScaler(
    inputCol="sepal_width_vec",
    outputCol="sepal_width_std",
    withMean=True,
    withStd=True
)

std_model = std_scaler.fit(scaled_df)
std_df = std_model.transform(scaled_df)
std_df.show(truncate=False)


+-----------------+----------------+-----------------+----------------+------+---------------+-------------------+----------------------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|sepal_width_vec|sepal_width_scaled |sepal_width_std       |
+-----------------+----------------+-----------------+----------------+------+---------------+-------------------+----------------------+
|4.9              |3.6             |1.4              |0.1             |0     |[3.6]          |[8.333333333333332]|[1.2450301512664121]  |
|4.4              |3.0             |1.3              |0.2             |0     |[3.0]          |[7.083333333333333]|[-0.13153881205026063]|
|5.1              |3.4             |1.5              |0.2             |0     |[3.4]          |[7.916666666666666]|[0.7861738301608543]  |
|5.0              |3.5             |1.3              |0.3             |0     |[3.5]          |[8.125]            |[1.0156019907136333]  |
|4.5              |2.3            

## Resumen / Conclusión

En este notebool se ve un montón de cosas que siempre terminamos haciendo al trabajar con datos en Spark y en big data:


Encontramos y arreglamos nulos

Borramos duplicados y columnas que estorbaban

Detectamos valores fuera de lo normal

Reorganizamos datos con pivot

Expandimos listas con explode

Normalizamos y estandarizamos para preparación de modelos

Todo esto te deja los datos limpios, ordenados y listos para análisis o machine learning